In [57]:
#Customers who left are -1, current customers are 1
#Males are 0, females are 1

#LOOK UP KURTOSIS, THE GUY DID IT ON MONTHLY TENURE, LOOKED COOL, IDK


#NOT SURE IF I SHOULD ONEHOTENCODE OR JUST TOKENIZE AS 0-4 FOR CREDIT CARD TYPE
    #I went with tkenizing 0-4 for now


#Open-to-Buy: The difference between the credit limit assigned to a cardholder account and the present balance on the account.
    #DO RATIO BETWEEN THIS AND CREDIT LIMIT




In [58]:
import pandas as pd
import numpy as np
from pathlib import Path

from imblearn.over_sampling import SMOTE
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.svm import SVC


In [59]:

starter_df = pd.read_csv(Path("c://users/ajcth/documents/github/bank_churn_project/Resources/BankChurners2.csv"))

starter_df = starter_df[starter_df != 'Unknown'].dropna()
starter_df.sample(5)


,CLIENTNUM,Attrition_Flag,Customer_Age,Gender,Dependent_count,Education_Level,Marital_Status,Income_Category,Card_Category,Months_on_book,...,Credit_Limit,Total_Revolving_Bal,Avg_Open_To_Buy,Total_Amt_Chng_Q4_Q1,Total_Trans_Amt,Total_Trans_Ct,Total_Ct_Chng_Q4_Q1,Avg_Utilization_Ratio,Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_1,Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_2
6385,789319458,Existing Customer,48,M,5,Uneducated,Single,$60K - $80K,Blue,38,...,22639.0,1481,21158.0,0.703,4219,65,0.477,0.065,0.000203,0.99980
3409,806656083,Existing Customer,48,F,2,Graduate,Married,$40K - $60K,Blue,43,...,9559.0,1514,8045.0,0.716,4064,70,0.556,0.158,0.000327,0.99967
9128,709171308,Existing Customer,30,M,1,High School,Married,$60K - $80K,Blue,16,...,15795.0,1742,14053.0,1.022,13173,96,0.846,0.110,0.000276,0.99972
7027,789634983,Existing Customer,59,M,1,Graduate,Married,$40K - $60K,Blue,53,...,4188.0,2200,1988.0,0.867,4911,86,0.720,0.525,0.000052,0.99995
4143,714850158,Existing Customer,49,M,4,Graduate,Single,$80K - $120K,Blue,36,...,1963.0,1516,447.0,0.872,4150,75,0.744,0.772,0.000097,0.99990


In [60]:
starter_df['Education_Level'].value_counts()

Graduate         2591
High School      1653
Uneducated       1204
College           844
Post-Graduate     431
Doctorate         358
Name: Education_Level, dtype: int64

In [61]:
#Shorten column names so they display better
replacement_column_names = {'Attrition_Flag': 'Status',
        'Customer_Age': 'Age',
        'Education_Level': 'Education',
        'Dependent_count': 'Dependents',
        'Income_Category': 'Income',
        'Card_Category': 'Card Type',
        'Months_on_book': 'Tenure(month)'
        }
starter_df.rename(columns=replacement_column_names, inplace=True)

In [62]:
#Drop unknowns
starter_df = starter_df[starter_df.Marital_Status != 'Unknown']
starter_df = starter_df[starter_df.Education != 'Unknown']
starter_df = starter_df[starter_df.Income != 'Unknown']

#NOT USING RIGHT NOWReplace M/F with -1/1
#starter_df = starter_df.replace({'Gender' : {'M': '-1', 'F': '1'}})

#Replace M/F with 0,1
starter_df = starter_df.replace({'Gender' : {'M': 0, 'F': 1}})

#Replace customer's status to 0 if they left and 1 if they stayed
starter_df = starter_df.replace({'Status' : {'Attrited Customer': -1, 'Existing Customer': 1}})





In [63]:
"""
use feature creation to make a ratio between dependents and income sources
consider someone divorced as Single. only 10% of divorced people receive alimony payments
        either a 1 (single/divorced) or a 2 (married)
        divide that by the amount of dependents plus one, which represents how many people you take care of
                i.e. a single person with no dependents only takes care of themself, so they are a 1. 
                """
starter_df = starter_df.replace({'Marital_Status' : {'Divorced': 1, 'Single': 1, 'Married' : 2}})
starter_df['MaritalDependentRatio'] = ((starter_df['Marital_Status'] / (starter_df['Dependents'] + 1)) / 2).round(2)
starter_df.drop(columns=['Marital_Status', 'Dependents'], inplace=True)

#Replace card categories from Blue/Silver/Gold/Plat to 0/1/2/3/4
starter_df = starter_df.replace({'Card Type' : {'Blue': 0, 'Silver': 1, 'Gold' : 3, 'Platinum' : 4}})


#Drop irrelevant column
starter_df = starter_df.drop(columns='CLIENTNUM')

In [64]:
#use feature creation to make a ratio between avg open to buy and credit limit

starter_df['CreditUsage'] = (starter_df['Avg_Open_To_Buy'] / starter_df['Credit_Limit']).round(2)
starter_df.drop(columns=['Avg_Open_To_Buy', 'Credit_Limit'], inplace=True)

In [65]:
#use feature creation to make a ratio between transaction amount and trans count
starter_df['AvgTransValue'] = (starter_df['Total_Trans_Amt'] / starter_df['Total_Trans_Ct']).round(2)
starter_df.drop(columns=['Total_Trans_Amt', 'Total_Trans_Ct'], inplace=True)

In [66]:
"""
There education categories are vague/ambiguous 
In 2021, the highest level of education of the population age 25 and older in the United States was distributed as follows: 
    8.9% had less than a high school diploma or equivalent.
        our dataset shows 17% uneducated
    27.9% had high school graduate as their highest level of school completed. 
        our dataset shows 23.3%
    14.9% had completed some college but not a degree.
        ours only has an ambiguous 'graduate' as 36.6%. could be this, could mean post-grad, term is used interchangeably.
    10.5% had an associate degree as their highest level of school completed.
        our dataset has no distinction between associate/undergrad. just says 'college'. ours shows 11.9% for college
    23.5% had a bachelor’s degree as their highest degree.
        ours shows 11.9% for college
    14.4% had completed an advanced degree such as a master’s degree, professional degree or doctoral degree. 
        ours shows 6.1% post-graduate, 5.1% doctorate, and the ambiguous 36.6% 'graduate'ArithmeticError
"""

counts = starter_df['Education'].value_counts()
percs = starter_df['Education'].value_counts(normalize=True).mul(100).round(1).astype(str) + '%'
pd.concat([counts,percs], axis=1, keys=['count', 'percentage'])

,count,percentage
Graduate,2591,36.6%
High School,1653,23.3%
Uneducated,1204,17.0%
College,844,11.9%
Post-Graduate,431,6.1%
Doctorate,358,5.1%


In [67]:
'''
It's hard to tell what they mean by 'graduate'. Unfortunately, its the bulk of our data, and dropping it will hurt our sample size. 
    The existence of a 'college' column suggests that graduate could mean beyond college. 
        Alternatively, it could mean you attended college but did not graduate, and 'graduate' represents standard undergrads.
Its a high amount of the data (36.6%), so its likely a mix of multiple categories not included, i.e. trade schools, associate degrees, dropouts, specializations
With so little info on what Graduate means, I'm dropping it all. 
We could try to create a weighting system that roughly ranks the other columns based on years at school (?)
    Essentially:
        uneducated = 0, highschool=1, college=2, post-grad=3, doctorate=4

'''
#Change education to ranking system, drop graduate
starter_df = starter_df[starter_df.Education != 'Graduate']
starter_df = starter_df.replace({'Education' : {'Uneducated': 0, 'High School': 1, 'College' : 2, 'Post-Graduate' : 3, 'Doctorate' : 4}})

#Here is another method that highly simplifies the data but includes the Graduate data
#starter_df = starter_df.replace({'Education' : {'Uneducated': 0, 'High School': 0, 'College' : 1, 'Graduate' :  1, 'Post-Graduate' : 2, 'Doctorate' : 2}})

In [68]:
#use feature creation to make a ratio between age and tenure
starter_df['TenureByAge'] = (starter_df['Tenure(month)'] / starter_df['Age']).round(2)
starter_df.drop(columns=['Tenure(month)', 'Age'], inplace=True)

In [69]:
starter_df = starter_df.dropna()

In [70]:
#use OneHotEncoder for the income categories
'''
enc = OneHotEncoder(sparse=False)
categorical_variables = ['Income']
encoded_data = enc.fit_transform(starter_df[categorical_variables])

encoded_df = pd.DataFrame(
    encoded_data,
    columns = enc.get_feature_names(categorical_variables)
)

encoded_df = pd.concat(
    [
        starter_df,
        encoded_df
    ],
    axis=1
)
encoded_df.drop(columns=['Income'], inplace=True)
encoded_df = encoded_df.dropna()
'''


"\nenc = OneHotEncoder(sparse=False)\ncategorical_variables = ['Income']\nencoded_data = enc.fit_transform(starter_df[categorical_variables])\n\nencoded_df = pd.DataFrame(\n    encoded_data,\n    columns = enc.get_feature_names(categorical_variables)\n)\n\nencoded_df = pd.concat(\n    [\n        starter_df,\n        encoded_df\n    ],\n    axis=1\n)\nencoded_df.drop(columns=['Income'], inplace=True)\nencoded_df = encoded_df.dropna()\n"

In [71]:
#Rank the income as a 0-4 low-high
starter_df = starter_df.replace({'Income' : {'Less than $40K': 0, '$40K - $60K': 1, '$80K - $120K' : 2, '$60K - $80K' : 3, '$120K +' : 4}})

In [72]:
#Not positive I should be scaling this... need to come back and evaluate
#should I also scale avg trans value?
#scaler = StandardScaler()
#starter_df['Total_Revolving_Bal'] = scaler.fit_transform(starter_df[['Total_Revolving_Bal']])

In [73]:
#Remove columns that intimidate me

starter_df.drop(columns=['Total_Relationship_Count', #¿que?
'Contacts_Count_12_mon', #Not a clue
'Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_2', #Some kind of wizardry, incantations
'Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_1', #I didnt make this and idk what it do or how it do it
'Total_Amt_Chng_Q4_Q1', 'Total_Ct_Chng_Q4_Q1', 'Avg_Utilization_Ratio', #Same as it ever was
'Total_Revolving_Bal' #not sure how to use this yet
],
inplace=True
)
starter_df.sample(10)

,Status,Gender,Education,Income,Card Type,Months_Inactive_12_mon,MaritalDependentRatio,CreditUsage,AvgTransValue,TenureByAge
1557,1,0,2,3,0,1,1.00,0.39,35.23,0.82
2655,1,1,0,0,0,1,0.20,0.47,51.68,0.74
5931,-1,1,0,0,0,3,0.33,1.00,54.55,0.83
8454,1,1,0,0,0,2,0.33,0.32,52.83,0.67
2045,1,0,1,4,0,3,0.12,1.00,41.49,0.72
3658,1,0,0,2,0,3,0.12,0.96,48.88,0.66
3278,1,1,1,0,0,1,0.12,0.86,36.67,1.00
3104,1,0,1,2,0,2,0.08,0.93,48.70,0.78
6247,1,1,0,0,0,2,0.20,0.56,62.09,0.85
5841,1,1,1,1,0,2,0.25,0.17,59.66,0.80


In [74]:
y = starter_df['Status']
X = starter_df.drop(columns=['Status'])

In [75]:
#Check the value counts
y.value_counts()

 1    5968
-1    1113
Name: Status, dtype: int64

In [76]:
#Split features and target in to training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [77]:
#Use SMOTE to add synthetic data and balance our target feature value count
oversample = SMOTE()
X_train, y_train = oversample.fit_resample(X_train, y_train)

In [78]:
#Choose a model
#clf = SVC()
#clf = LogisticRegression()
clf = RandomForestClassifier(n_estimators= 2000)

In [79]:
#Fit the model on the training data
clf.fit(X_train, y_train)

RandomForestClassifier(n_estimators=2000)

In [80]:
#Make predictions on the test data
test_predictions = clf.predict(X_test)

In [81]:
# Evaluate the model's performance
accuracy = accuracy_score(test_predictions, y_test)
print("Accuracy: {:.2f}%".format(accuracy * 100))

Accuracy: 81.37%


In [82]:
#Create a dataframe showing predictions vs test
comparison = pd.DataFrame({"Testing Data Predictions": test_predictions,
    "Testing Data Actual Targets": y_test})
comparison = comparison.sort_index(ascending=True)
comparison.tail(5)

,Testing Data Predictions,Testing Data Actual Targets
10097,1,1
10098,-1,-1
10108,1,-1
10112,1,-1
10117,1,1


In [83]:
#Generate a test matrix
test_matrix = confusion_matrix(y_test, test_predictions)
print(test_matrix)

[[ 106  171]
 [ 159 1335]]


In [84]:
# Create a testing classifiction report
testing_report = classification_report(y_test, test_predictions)

# Print the testing classification report
print(testing_report)

              precision    recall  f1-score   support

          -1       0.40      0.38      0.39       277
           1       0.89      0.89      0.89      1494

    accuracy                           0.81      1771
   macro avg       0.64      0.64      0.64      1771
weighted avg       0.81      0.81      0.81      1771

